Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [11]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [12]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [13]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [14]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [15]:
vocabulary_size_base = len(string.ascii_lowercase) + 1 # [a-z] + ' '
vocabulary_size = vocabulary_size_base ** 2

first_letter = ord(string.ascii_lowercase[0])

def char2id0(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0

def char2id(char):
  return char2id0(char[0]) * vocabulary_size_base + char2id0(char[1])
  
def id2char0(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

def id2char(dictid):
  return id2char0(dictid//vocabulary_size_base) + id2char0(dictid%vocabulary_size_base)

print(char2id('ab'), char2id('za'), char2id(' a'), char2id('ï '))
print(id2char(1), id2char(26*27+0), id2char(0))

Unexpected character: ï
29 703 1 0
 a z    


Function to generate a training batch for the LSTM model.

In [16]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    # batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    # Changed: at this time, we only need a list, parameters can be find from embeddings
    batch = list()
    for b in range(self._batch_size):
      # batch[b, char2id(self._text[self._cursor[b]:self._cursor[b]+2])] = 1.0
      # Changed: just append the next bigram word to the list
      batch.append(char2id(self._text[self._cursor[b]:self._cursor[b]+2]))
      self._cursor[b] = (self._cursor[b] + 1) % (self._text_size - 1)
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches # (num_unrollings + 1, batch_size, vocabulary_size)

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  # s = [''] * batches[0].shape[0]
  # Changed: since now we have a list, the length is len(batches)
  s = [''] * len(batches)
  for b in batches:
    # s = [''.join(x) for x in zip(s, characters(b))]
    # Changed: just convert id to char is enough
    s = [''.join(x) for x in zip(s, [id2char(c) for c in b])]
  s = [bigramstringtonormal(i) for i in s]
  return s

def bigramstringtonormal(bigramstring):
  ret = [bigramstring[0]]
  for i in range((len(bigramstring)-2)//2):
    ret.append(bigramstring[i*2+1] if bigramstring[i*2+1] == bigramstring[i*2+2] else '[%s]'%(bigramstring[i*2+1:i*2+3], ))
  ret.append(bigramstring[-1])
  return ''.join(ret)

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(valid_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))

[' an']
['ons anarchis', 'when militar', 'lleria arche', ' abbeys and ', 'married urra', 'hel and rich', 'y and liturg', 'ay opened fo', 'tion from th', 'migration to', 'new york oth']
['ists advocat', 'ary governme', 'hes national', 'd monasterie', 'raca princes', 'chard baer h', 'rgical langu', 'for passenge', 'the national', 'took place d', 'ther well kn']
['ana']


In [17]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [18]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ifcox = tf.Variable(tf.truncated_normal([vocabulary_size, 4 * num_nodes], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    # all_gates = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
    # Changed: instead of calculating the multiplication, try embedding_lookup this time
    all_gates = tf.nn.embedding_lookup(ifcox, i) + tf.matmul(o, ifcom) + ifcob
    input_gate = tf.sigmoid(all_gates[:, 0:num_nodes])
    forget_gate = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
    update = all_gates[:, 2*num_nodes:3*num_nodes]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(all_gates[:, 3*num_nodes:])

    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      # tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
      # Changed: embedding_lookup by the indices
      tf.placeholder(tf.int32, shape=[batch_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs: # i.shape: (batch_size, vocabulary_size)
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b) # (batch_size * num_unrollings, vocabulary_size)
    loss = tf.reduce_mean(
      # tf.nn.softmax_cross_entropy_with_logits(
      # Changed: since the label is just indices, we changed to the sparse version API
      tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  # sample_input = tf.placeholder(tf.int64, shape=[1, vocabulary_size])
  # Changed: just like the train_input, change it to int and (1, )
  sample_input = tf.placeholder(tf.int64, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [21]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i] # (batch_size, vocabulary_size)
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      # Changed: to calculate perplexity, we need ont-hot encoded labels, so, convert it
      noembed_labels = np.zeros(predictions.shape)
      for i, j in enumerate(labels):
        noembed_labels[i, j] = 1.0
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, noembed_labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          # Changed: convert to embedding
          feed = [np.argmax(feed)]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
            # Changed: convert to embedding
            feed = [np.argmax(feed)]
          print(bigramstringtonormal(sentence))
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        # Changed: to calculate perplexity, we need ont-hot encoded labels, so, convert it
        labels = np.zeros((1, vocabulary_size))
        labels[0, b[1]] = 1.0
        valid_logprob = valid_logprob + logprob(predictions, labels)
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.591677 learning rate: 10.000000
Minibatch perplexity: 729.00
l[wa][ul][ d][lf][kj][jw]g[dt][ks][ r][nw][d ][ad][he][tn][ap][zd][kn]j[ae]c[oa][c ][aq][fv][ o][xn][kw][uz][ba][pm][qr][ga]y[ij]b[ t][jg][al][nx][ub][ d][ga][th][wt][vf][fj][rx][pa][iy][l ][hc][fy][me][py][qm][li][ji][mj][nr][wg][tq][ms][ms][bv][jo][sd][ce][ r][tj][zx][cf][fm][xs][ef][mz][vj][ef]kc
u[ut][rm][s ][us][pt][lj][ol][cw][mw][ib][ c][vr][id][la][yk][s ][bk][tg][ki][ig][xt][sn][uf][od][gh][lz][wb][gw][rq][yj][lx][ld][iv][cn][nd][xg][kj][ o][ni][uq][ac][gl][hg][qw][uh][ap][ h][sc][kz][df][od][wm][cm][xz][zv][pb][po][pi][nb][ri][fh][hm][yd][vt][ko][pl][ws][ c][yi][yr][pr][gk][fe][sa][ea][tx][cf][jh][kg]w
n[fa][pf][wo][fv][ti][ e][l ][lh][xd][lw][ p][uj][hb][eg][xj][wl][du][km][ri][nu][m ][mp][gd][sq][sv][ r][ko][ r][xn][q ][cr][ a][nm][be][vk][hz][xg][wv]e[wa][ip][wf][mz][cq][od][um][db][lj][ec][gu][js][ b]c[kw][hy][ab][ia][bj][iu][gu][pq][oy][nt]u[bi][ny][ e][pi][to][lq][dx][dc][

KeyboardInterrupt: 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---